In [101]:
import time as time
import numpy as np
import pandas as pd

In [102]:
def removeNaN(data):
    resultList = [] # return data without NaN
    for i in range(0,len(data)):
        resultList.append(list(data.iloc[i].dropna()))
    return resultList

In [103]:
def allItem(data):
    resultList=[] # return list of all possible items
    for transaction in data:
        for item in transaction:
            if not [item] in resultList:
                resultList.append([item])
    resultList=np.sort(resultList)
    return list(map(frozenset, resultList))

In [104]:
def frequentItemset(data, itemset):
    countList = {}  # count each candidate in our dataset
    for d in data:
        for i in itemset:
            if i.issubset(d):
                if not i in countList: 
                    countList[i]=1
                else: 
                    countList[i] += 1                
    
    return countList

In [105]:
def minSupport(frequentItem, numItems, pctSupport = 0.2):
    resultList=[] # items that meets the minimum support
    supInfo = {} # dictionary that will contains support%
    
    # calculate the frequency
    for i in frequentItem:
        support = round(frequentItem[i]/numItems,2)
        supInfo[i] = support
        if support >= pctSupport:
            resultList.insert(0,i)
    return resultList, supInfo

In [106]:
def generateItemset(frequentItem, n): 
    resultList = []
    #print(frequentItem)
    for i in range(len(frequentItem)):
        for j in range(i+1, len(frequentItem)): 
            item1 = list(frequentItem[i])[:n-2]
            item1.sort()
            item2 = list(frequentItem[j])[:n-2]
            item2.sort()
            if item1==item2: #if first k-2 elements are equal
                #print(frequentItem[i] | frequentItem[j])
                resultList.append(frequentItem[i] | frequentItem[j]) 
    
    return resultList

In [107]:
def apriori(dataSet, pctSupport = 0.5): # minimum support is defaulted to 50%
    # give us all items in our data
    itemSet = allItem(dataSet)
    item = list(map(set, dataSet)) # map our dataset into list
    numItems = len(item)
    
    frequentItem = frequentItemset(item,itemSet)
    frequentItem,supInfo = minSupport(frequentItem,numItems,pctSupport)
    frequentItem=[frequentItem]; n = 2

    # it will generate the i-itemset, calculate the frequenty and use to identify minimum support
    while (len(frequentItem[n-2]) > 0):
        nItemSet = generateItemset(frequentItem[n-2], n) #creates i-itemset that met our minimum support
        nFrequentItem = frequentItemset(item, nItemSet)
        nFrequentItem, nSupInfo=minSupport(nFrequentItem, numItems, pctSupport)
        
        supInfo.update(nSupInfo)
        frequentItem.append(nFrequentItem)
        n += 1
        
    return frequentItem, supInfo

In [108]:
def generateRules(frequentSet, suppData, pctConf=0.7):  #supportData is a dict coming from scanD
    resultList = []
    for i in range(0,len(frequentSet)):
        for freqSet in frequentSet[i]:
            itemList = []
            for item in freqSet:
                itemList.append(frozenset([item]))
            
            #print(itemList)
            if (i > 1):
                rulePerm(freqSet, itemList, suppData, resultList, pctConf)
            else:
                minConf(freqSet, itemList, suppData, resultList, pctConf)
    return resultList  

In [109]:
def minConf(X, item, suppData, returnList, pctConf=0.7):
    prunedItem = [] 
    for Y in item:
        
        # the confidence is calculated as Support(X and Y)/Support(X)
        pct = suppData[X]/suppData[X-Y] 
        
        if pct >= pctConf: 
            pct = round(pct,2) # round to two decimal places
            returnList.append((X-Y, Y, pct)) # X -> Y; confidence  
            prunedItem.append(Y)
    
    return prunedItem

In [110]:
def rulePerm(X, itemList, suppData, resultList, pctConf=0.7): # identify permutation for all the assoication rule
    num = len(itemList[0])
    
    # capture the confidence on a data set {B C D} where B C -> D and B D -> C
    if (len(X) > (num + 1)): 
        X_num= generateItemset(itemList, num+1)
        
        # calculate the confidence and retrieve ones that meets our threshold
        X_num = minConf(X, X_num, suppData, resultList, pctConf)
        if (len(X_num) > 1):
            rulePerm(X, X_num, suppData, resultList, pctConf)

In [111]:
 def printRule(fileName, rules, suppData):
    result = ''
    print('Association Rules (%s rules):' % len(rules))
    for rule in rules:
        # burgers --> ground beef [20.0%, 80.0%] 
        assocRule = ', '.join(rule[0]) + ' --> '  + ', '.join(rule[1])
        suppPct = str(100*suppData[frozenset.union(rule[0],rule[1])])
        confPct = str(round(100*rule[2],2))
        result = '\t' + assocRule + ' [' + suppPct + '%, ' + confPct + '%]'
        print(result)

In [112]:
def aprioriAlgorithm(fileName, pctMinSup, pctMinConf):
    data=pd.read_csv(fileName)
    items=removeNaN(data)
    # run apriori algorithm
    frequentSet, suppData = apriori(items,pctMinSup)
    frequentSet = frequentSet[1:] # 1-itemsets becuase no rule is generated
    
    # generate association rule
    rules= generateRules(frequentSet,suppData, pctMinConf)
    printRule(fileName, rules, suppData) # print all rules

In [113]:
def combinationItem(items, num): 
    items = list(items) 
    numItems = len(items) 
    if num > numItems: 
        return
    index = np.arange(num) 
    # return our first combination
    yield list(items[i] for i in index) 
    
    
    while True:  # loop till no combination is available
        for i in reversed(range(num)): 
            if index[i] != i + numItems - num: 
                break
        else: 
            return
        
        index[i] += 1
        
        for j in range(i + 1, num): 
            index[j] = index[j-1] + 1            
        
        yield list(items[i] for i in index) 

In [114]:
# remove nested list and combine into one list
def removeNestedList(thisList,result = []): 
    for l in thisList: 
        if type(l) == list: 
            removeNestedList(l, result) 
        else: 
            result.append(l) 
    return result

In [115]:
def bruteForce(fileName, pctMinSup, pctMinConf):
    data = pd.read_csv(fileName)  
    items=removeNaN(data)
    itemSet = allItem(items)
    itemList = [list(item) for item in itemSet]
    output = removeNestedList(itemList,[])
    
    bruteForceItem = []
    bruteForceSup = {}
    
    k = 1 # number of itemset combination
    numItems = len(items)
    while True:
        # k combination 
        combined = [frozenset(x) for x in combinationItem(output, k)]
        
        # get frequent on each combination
        frequentItem = frequentItemset(items,combined)
        
        # filter out combination with our minimum support
        frequentItem,supInfo = minSupport(frequentItem,numItems,pctMinSup)
        
        if frequentItem == []: break # get out once k-itemset did not meet minimum support
        k = k+1 # increment k-itemset
        bruteForceSup.update(supInfo)
        bruteForceItem.append(frequentItem)

    bruteForceItem = bruteForceItem[1:]
    # run generate association rule
    rules= generateRules(bruteForceItem,bruteForceSup, pctMinConf)
    printRule(fileName, rules, bruteForceSup) # print out rules

In [117]:
minimum_support = .2
minimum_confidence = .7
files = ['./store_a_data.csv','./store_b_data.csv',
             './store_c_data.csv','./store_d_data.csv',
             './store_e_data.csv']

# we will loop per file and run Apiori Algorithm and Brute Force
# and capture the time and print out the association rules..
i = 1
for fileName in files:
    print('Database #%d - Filename: %s \n' % (i,fileName))
    print('Apriori Algorithm:')
    print('------------------')
    start_time = time.time() # store
    aprioriAlgorithm(fileName, minimum_support, minimum_confidence)
    print('Total time running: %s seconds' % str(time.time()-start_time))
    print()
    print('Brute Force:')
    print('------------------')
    start_time = time.time()
    bruteForce(fileName, minimum_support, minimum_confidence)
    print('Total time running: %s seconds' % str(time.time()-start_time))
    print()
    i=i+1

Database #1 - Filename: ./store_a_data.csv 

Apriori Algorithm:
------------------
Association Rules (1 rules):
	mineral water --> eggs [20.0%, 80.0%]
Total time running: 0.025922060012817383 seconds

Brute Force:
------------------
Association Rules (1 rules):
	mineral water --> eggs [20.0%, 80.0%]
Total time running: 0.0639810562133789 seconds

Database #2 - Filename: ./store_b_data.csv 

Apriori Algorithm:
------------------
Association Rules (2 rules):
	spaghetti --> tomatoes [20.0%, 80.0%]
	avocado --> chicken [20.0%, 100.0%]
Total time running: 0.011481761932373047 seconds

Brute Force:
------------------
Association Rules (2 rules):
	spaghetti --> tomatoes [20.0%, 80.0%]
	avocado --> chicken [20.0%, 100.0%]
Total time running: 0.046716928482055664 seconds

Database #3 - Filename: ./store_c_data.csv 

Apriori Algorithm:
------------------
Association Rules (2 rules):
	burgers --> ground beef [20.0%, 80.0%]
	spaghetti --> tomatoes [20.0%, 100.0%]
Total time running: 0.010782957077

Total time running: 10.703157186508179 seconds

Database #5 - Filename: ./store_e_data.csv 

Apriori Algorithm:
------------------
Association Rules (16 rules):
	cake --> pancakes [20.0%, 80.0%]
	spaghetti --> pasta [20.0%, 80.0%]
	pasta --> spaghetti [20.0%, 100.0%]
	cake --> brownies [25.0%, 100.0%]
	milk --> brownies [25.0%, 71.0%]
	ground beef --> eggs [20.0%, 100.0%]
	pancakes --> brownies [35.0%, 100.0%]
	brownies --> pancakes [35.0%, 70.0%]
	mineral water --> salmon [30.0%, 86.0%]
	salmon --> mineral water [30.0%, 86.0%]
	soup --> brownies [30.0%, 75.0%]
	cake, pancakes --> brownies [20.0%, 100.0%]
	cake, brownies --> pancakes [20.0%, 80.0%]
	brownies, milk --> soup [20.0%, 80.0%]
	soup, milk --> brownies [20.0%, 100.0%]
	soup, pancakes --> brownies [20.0%, 100.0%]
Total time running: 0.015034198760986328 seconds

Brute Force:
------------------
Association Rules (16 rules):
	cake --> pancakes [20.0%, 80.0%]
	spaghetti --> pasta [20.0%, 80.0%]
	pasta --> spaghetti [20.0%, 100.0%